In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import catboost

In [87]:
def is_titled(x):
    firstlet=[]
    for i in x:
        if i[0:1].isupper()!=i[1:2].isupper():
            i=1
        else:
            i=0
        firstlet.append([i])
    return firstlet

def length(x):
    length=[]
    for i in x:
        length.append([len(i)])
    return length

def syls(x):
    syl=''
    syls=[]
    cons='бвгджзйклмнпрстфхцчшщьъ'
    vow='ауоыиэяюёе'
    for i in (x[::-1]+'|').lower():
        #print(i)
        if i in cons:
            syl=i+syl
        elif i in vow:
            syls+=[(i+syl).strip()]
            syl=''
        elif i!='|':
            continue
        else:
            syls+=[(syl).strip()]
            break
    syls.reverse()
    return syls
    
def words_by_syls(x,f=None,t=None):
    x_by_syls=[]    
    for i in x:
        x_by_syls.append(str(' '.join(syls(i)[f:t])))
    return x_by_syls

def matches_pattern (x):
    matches=[]
    for i in x:
        if re.fullmatch('.*((ко)|(ов)|(ян)|(ев))$',i):
            i=1
        else:
            i=0
        matches.append([i])
    return matches

def words_by_let(x,f=None,t=None):
    x_by_let=[]
    cons='бвгджзйклмнпрстфхцчшщьъ'
    vow='ауоыиэяюёе'
    lets=[]
    for i in x:
        for j in i:
            if j.lower() in cons:
                i=i.replace(j,'C')
            elif j.lower() in vow:
                i=i.replace(j,'Г')
            else:
                i=i.replace(j,'Х')
        lets.append(i)
    return lets

In [88]:
surnames=pd.read_csv(r"/kaggle/input/secondname/linear_train.txt", names=['Word','Lastname'],sep=',')
test=pd.read_csv(r"/kaggle/input/secondname/linear_test.txt", names=['Word'],sep=',')

In [89]:
print(surnames['Word'][:10])
print(words_by_syls(surnames['Word'][:10],-1))
print(is_titled(surnames['Word'][:10]))
print(matches_pattern(surnames['Word'][:10]))
print(words_by_let(surnames['Word'][:10]))

0    Аалтонен
1         Аар
2       Аарон
3       ААРОН
4      Аарона
5      Аарона
6      Аароне
7     Ааронов
8      Аахена
9     Абабков
Name: Word, dtype: object
['ен', 'ар', 'он', 'он', 'а', 'а', 'е', 'ов', 'а', 'ов']
[[1], [1], [1], [0], [1], [1], [1], [1], [1], [1]]
[[0], [0], [0], [0], [0], [0], [0], [1], [0], [1]]
['ГГCCГCГC', 'ГГC', 'ГГCГC', 'ГГCГC', 'ГГCГCГ', 'ГГCГCГ', 'ГГCГCГ', 'ГГCГCГC', 'ГГCГCГ', 'ГCГCCГC']


In [90]:
def get_in_shape_by_syl(x,ofst,param,param2):
    global return_features
    global vectorizer1
    global vectorizer2
    global offset
    offset=ofst
    vectorizer1=CountVectorizer(**param)
    last_syls1 = vectorizer1.fit_transform([i[-offset:] for i in x])
    last_syls1 = last_syls1.toarray()
    vectorizer2=CountVectorizer(**param2)
    last_syls2 = vectorizer2.fit_transform(words_by_let(x))
    last_syls2 = last_syls2.toarray()
    return np.hstack((is_titled(x),last_syls1,last_syls2))

def get_in_shape_by_syl2(x,strt,ed,param,param2):
    global return_features
    global vectorizer1
    global vectorizer2
    global start
    global end
    start=strt
    end=ed
    vectorizer1=CountVectorizer(**param)
    last_syls1 = vectorizer1.fit_transform(words_by_syls(surnames['Word'],start,end))
    last_syls1 = last_syls1.toarray()
    vectorizer2=CountVectorizer(**param2)
    last_syls2 = vectorizer2.fit_transform(words_by_let(x))
    last_syls2 = last_syls2.toarray()
    return np.hstack((is_titled(x),last_syls1,last_syls2))

In [91]:
def get_in_shape_by_syl_fin(x):
    last_syls1 = vectorizer1.transform([i[-offset:] for i in x])
    last_syls1 = last_syls1.toarray()
    last_syls2 = vectorizer2.transform(words_by_let(x))
    last_syls2 = last_syls2.toarray()
    return np.hstack((is_titled(x),last_syls1,last_syls2))

def get_in_shape_by_syl_fin2(x):
    last_syls1 = vectorizer1.transform(words_by_syls(surnames['Word'],start,end))
    last_syls1 = last_syls1.toarray()
    last_syls2 = vectorizer2.transform(words_by_let(x))
    last_syls2 = last_syls2.toarray()
    return np.hstack((is_titled(x),last_syls1,last_syls2))

In [ ]:
?catboost.CatBoostClassifier

In [99]:
#algor = LogisticRegression(penalty='l1',C=1, max_iter=100)
algor=catboost.CatBoostClassifier(num_trees=1000, depth=3)
x_shaped=get_in_shape_by_syl(x=surnames['Word'],
                             ofst=3,
                             param={'analyzer':'char_wb',
                                    'max_features':1000,
                                    'min_df':0.001,
                                    'ngram_range':(3,3),
                                    'strip_accents':'unicode'},
                             param2={'analyzer':'word',
                                    'max_features':1000,
                                    'min_df':0.001,
                                    'strip_accents':'unicode'})
print('Dense matrix shape', x_shaped.data.shape)
#for i in range(10):
    #print(surnames['Word'][i],x_shaped[i])

Dense matrix shape (101408, 641)


In [ ]:
#algor = LogisticRegression(penalty='l1',C=1, max_iter=1000)
#algor=catboost.CatBoostClassifier(num_trees=1000, depth=3)
#
x_shaped=get_in_shape_by_syl2(x=surnames['Word'],
                             strt=-2, ed=None,
                             param={'analyzer':'word',
                                    'max_features':1000,
                                    'min_df':0.001,
                                    'ngram_range':(1,2),
                                    'strip_accents':'unicode'})
print('Dense matrix shape', x_shaped.data.shape)
#for i in range(10):
    #print(surnames['Word'][i],x_shaped[i])

In [95]:
num_2_words = {
    v: k
    for k, v in vectorizer2.vocabulary_.items()
}
num_2_words

{141: 'гcгccгc',
 146: 'гcгcгc',
 149: 'гcгcгcгc',
 148: 'гcгcгcг',
 139: 'гcгc',
 142: 'гcгccгcг',
 147: 'гcгcгccг',
 120: 'гccгc',
 126: 'гccгcг',
 127: 'гccгcгc',
 130: 'гccгcгcг',
 131: 'гccгcгcгc',
 121: 'гccгccг',
 128: 'гccгcгccг',
 145: 'гcгcг',
 143: 'гcгccгcгc',
 144: 'гcгccгcгг',
 150: 'гcгcгcгcг',
 140: 'гcгccг',
 132: 'гccгcгcгcг',
 133: 'гccгcгcгcгc',
 122: 'гccгccгc',
 123: 'гccгccгcг',
 124: 'гccгccгcгc',
 138: 'гcг',
 137: 'гccгcгг',
 152: 'гcгcгг',
 119: 'гccг',
 129: 'гccгcгccгc',
 125: 'гccгccгcгг',
 136: 'гccгcгcггc',
 135: 'гccгcгcгг',
 134: 'гccгcгcгcгг',
 5: 'ccгcccгc',
 151: 'гcгcгcгг',
 30: 'ccгcгг',
 118: 'гcccгcг',
 1: 'cccгcгc',
 0: 'cccгcг',
 31: 'cгc',
 71: 'cгcг',
 109: 'cгcггc',
 41: 'cгccг',
 78: 'cгcгccг',
 79: 'cгcгccгc',
 89: 'cгcгcгc',
 72: 'cгcгc',
 96: 'cгcгcгcг',
 88: 'cгcгcг',
 97: 'cгcгcгcгc',
 108: 'cгcгг',
 42: 'cгccгc',
 53: 'cгccгcг',
 54: 'cгccгcгc',
 82: 'cгcгccгcг',
 111: 'cгcггcгc',
 74: 'cгcгcccг',
 107: 'cгcгcггc',
 106: 'cгcгcгг',
 

In [98]:
arr = cross_val_score(algor, x_shaped, surnames['Lastname'], cv=2, scoring='roc_auc')
print(arr)
print(np.mean(arr))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.88419968 0.87068492 0.85384537 0.86039721]
0.8672817939368871


penalty='l1',C=1, max_iter=1000
0.8659940076199003
penalty='l1',C=1, max_iter=100
0.8659915839197074
'ngram_range':(2,3)
0.8672696984422832
'max_features':1500
0.8678805938316401

In [ ]:
algor.fit(x_shaped,surnames['Lastname'])

In [ ]:
for i in range(10):
    print(test['Word'][i],get_in_shape_by_syl_fin(test['Word'][i]))

In [ ]:
y_test=algor.predict_proba(get_in_shape_by_syl_fin(test['Word']))[:,1]

In [ ]:
for i in range(10):
    print(test['Word'][i],':',y_test[i])

In [ ]:
algor.predict_proba(get_in_shape_by_syl_fin(['Петров']))[:,1]

In [ ]:
output = pd.DataFrame({'Id': range(len(y_test)), 'Answer': y_test})
output.to_csv('Submissions.csv', index=False)